In [1]:
import os

import dotenv
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

dotenv.load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# RAG against my machine

In [5]:
# Parameters

# Embedding model for retrieval
# embed_id = "sentence-transformers/all-mpnet-base-v2"
# embed_id = "sentence-transformers/all-MiniLM-L6-v2"  # Faster model
embed_id = "paraphrase-multilingual-MiniLM-L12-v2"  # Multilingual model, including Dutch
embed_model_kwargs = {"device": "cpu"}
embed_encode_kwargs = {"normalize_embeddings": False}

# Vector store
chroma_db = "./chroma_db"
vectorstore_search_kwargs = {"k": 6}

# LLM model for generation
# llm_id = "Rijgersberg/GEITje-7B"
# llm_id = "GroNLP/bert-base-dutch-cased"
# llm_id = "./ov_model_dir"
llm_id = "./GEITje-7B-chat-v2"
llm_pipeline_kwargs = {"max_new_tokens": 100, "return_full_text": False}  # TODO: extend with more generation parameters
llm_model_kwargs = {"ov_config": {"KV_CACHE_PRECISION": "u8",
                                  "DYNAMIC_QUANTIZATION_GROUP_SIZE": "32",
                                  "PERFORMANCE_HINT": "LATENCY",
                                  "NUM_STREAMS": "1",
                                  "CACHE_DIR": ""}}

In [6]:
# Load embedding model for retrieval
embed_model = HuggingFaceEmbeddings(model_name=embed_id, model_kwargs=embed_model_kwargs, encode_kwargs=embed_encode_kwargs)

# Create the vectorstore, load from disk
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embed_model)

# Convert vectorstore to retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs=vectorstore_search_kwargs)

In [8]:
question = "Welke kerncentrale is recent buiten bedrijf gesteld?"

In [9]:
# Retrieve documents as example
retrieved_docs = retriever.invoke(question)

print(f"{len(retrieved_docs)} retrieved docs")
print("Contents:")
for i, doc in enumerate(retrieved_docs):
    print(f"Doc {i}: {doc.page_content[:200]}")

6 retrieved docs
Contents:
Doc 0: title: Enige Litouwse kerncentrale dicht
content: De enige kerncentrale van Litouwen is oudjaarsavond om 23.00 uur buiten gebruik gesteld. Dat verliep zonder problemen, aldus de directeur. Litouwen be
Doc 1: Olympische organisatie, heeft 650 bedrijven gevraagd om het vervoer van werknemers terug te dringen, maar daarop is nog niet veel respons gekomen. Het is business as usual voor veel bedrijven. Andere 
Doc 2: Skidmore, Owens en Meril (SOM) dat gespecialiseerd is in wolkenkrabbers. Zij ontwierpen onder andere de Sears Tower in Chicago en de Russia Tower in Moskou. Woestijnbloem Het ontwerp van de toren is g
Doc 3: content: De luchthaven Schiphol koopt nog eens zestig bodyscanners. Er staan er nu zestien, maar de controles worden sterk uitgebreid. De eerste twintig nieuwe scanners staan er volgende week. De aanl
Doc 4: Het betrof echter slechts een mondeling akkoord. Toen DSB failliet ging en curatoren het bewind overnamen, erkenden zij de regeling ni

In [10]:
# prompt = ChatPromptTemplate.from_messages(
#     [("human", "Je bent een assistent voor het beantwoorden van vragen. Gebruik de volgende stukjes opgehaalde context om de vraag te beantwoorden. Als je het antwoord niet weet, zeg dan gewoon dat je het niet weet. Gebruik maximaal drie zinnen en houd het antwoord beknopt.\nVraag: {question} \nContext: {context} \nAntwoord:")])

# Input prompt
prompt = ChatPromptTemplate.from_messages(
    [("human", "<s> Je bent een assistent voor het beantwoorden van vragen. Gebruik de volgende stukjes opgehaalde context om de vraag te beantwoorden. Als je het antwoord niet weet, zeg dan gewoon dat je het niet weet. Gebruik maximaal drie zinnen en houd het antwoord beknopt. </s> Vraag: {question} \nContext: {context} \nAntwoord: ")])

example_messages = prompt.invoke({"context": "filler context", "question": "filler question"}).to_messages()
print(example_messages)

[HumanMessage(content='<s> Je bent een assistent voor het beantwoorden van vragen. Gebruik de volgende stukjes opgehaalde context om de vraag te beantwoorden. Als je het antwoord niet weet, zeg dan gewoon dat je het niet weet. Gebruik maximaal drie zinnen en houd het antwoord beknopt. </s> Vraag: filler question \nContext: filler context \nAntwoord: ')]


In [3]:
# Create the LLM used for generation
llm = HuggingFacePipeline.from_model_id(model_id=llm_id,
                                        task="text-generation",
                                        backend="openvino",
                                        model_kwargs=llm_model_kwargs,
                                        pipeline_kwargs=llm_pipeline_kwargs)  # TODO: Add device_map="auto" once device= is removed
llm

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


C:\Users\denbe\AppData\Roaming\Python\Python39\site-packages\transformers\utils\import_utils.py:519: FutureWarning: `is_torch_tpu_available` is deprecated and will be removed in 4.41.0. Please use the `is_torch_xla_available` instead.
  warnings.warn(
Compiling the model to CPU ...


In [11]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [13]:
print(question)

Welke kerncentrale is recent buiten bedrijf gesteld?


In [14]:
answer = rag_chain.invoke(question)

print(answer)



De FNV-bond heeft vanmorgen bij de Groningse vleesverwerker Beusmeat in Leek actie gevoerd tegen de uitbuiting van 150 Poolse uitzendkrachten. De bond deelde bij de poort pamfletten uit om de Polen te steunen. Volgens de bond krijgen ze minder dan het minimumloon en hebben ze een 50
